In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf
import matplotlib.dates as mdates
import datetime
import pandas_datareader as datareader
import datetime
import yfinance as yf
import tqdm

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

pio.renderers.default = "notebook"
pio.templates.default = "plotly_dark"
import gc
import warnings

warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = [12, 8]

In [40]:
fff = pd.read_csv("fff3_daily.CSV", skiprows=3, index_col=0)

In [41]:
fff

,Mkt-RF,SMB,HML,RF
19260701,0.10,-0.25,-0.27,0.009
19260702,0.45,-0.33,-0.06,0.009
19260706,0.17,0.30,-0.39,0.009
19260707,0.09,-0.58,0.02,0.009
19260708,0.21,-0.38,0.19,0.009
...,...,...,...,...
20221125,-0.02,0.22,0.44,0.014
20221128,-1.55,-0.40,-0.20,0.014
20221129,-0.18,0.10,1.03,0.014
20221130,3.12,0.36,-2.07,0.014


In [42]:
fff = fff.dropna()

In [43]:
fff.index = fff.index.astype(str)

In [44]:
fff.index

Index(['19260701', '19260702', '19260706', '19260707', '19260708', '19260709',
       '19260710', '19260712', '19260713', '19260714',
       ...
       '20221116', '20221117', '20221118', '20221121', '20221122', '20221123',
       '20221125', '20221128', '20221129', '20221130'],
      dtype='object', length=25378)

In [45]:
def convert_to_datetime(x):
    return datetime.datetime.strptime(x, "%Y%m%d")

In [46]:
pd.Series(fff.index).apply(convert_to_datetime)

0       1926-07-01
1       1926-07-02
2       1926-07-06
3       1926-07-07
4       1926-07-08
           ...    
25373   2022-11-23
25374   2022-11-25
25375   2022-11-28
25376   2022-11-29
25377   2022-11-30
Length: 25378, dtype: datetime64[ns]

## Using BS4

In [47]:
from bs4 import BeautifulSoup
import requests

In [48]:
url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [81]:
para = soup.find("p", style="width: 737px;")

In [82]:
links = para.find_all("a")

In [83]:
links

[<a name="Research"></a>,
 <a href="Data_Library/changes_crsp.html"><font color="#336699" size="2">Changes in CRSP
 										Data</font></a>,
 <a href="ftp/F-F_Research_Data_Factors_TXT.zip"><font color="#336699" face="Times New Roman, Times, serif" size="1">TXT</font></a>,
 <a href="ftp/F-F_Research_Data_Factors_CSV.zip"><font color="#336699" face="Times New Roman, Times, serif" size="1">CSV</font></a>,
 <a href="Data_Library/f-f_factors.html"><font color="#336699" face="Times New Roman, Times, serif" size="1">Details</font></a>,
 <a href="Data_Library/f-f_factors_archive.html"><font color="#336699" face="Times New Roman, Times, serif" size="1">Historical Archives</font></a>,
 <a href="ftp/F-F_Research_Data_Factors_weekly_TXT.zip"><font color="#336699" face="Times New Roman, Times, serif" size="1">TXT</font></a>,
 <a href="ftp/F-F_Research_Data_Factors_weekly_CSV.zip"><font color="#336699" face="Times New Roman, Times, serif" size="1">CSV</font></a>,
 <a href="Data_Library/f-f_factor

In [84]:
links = pd.Series(links)
links

0                                                  []
1     [[Changes in CRSP\r\n\t\t\t\t\t\t\t\t\t\tData]]
2                                             [[TXT]]
3                                             [[CSV]]
4                                         [[Details]]
5                             [[Historical Archives]]
6                                             [[TXT]]
7                                             [[CSV]]
8                                         [[Details]]
9                                             [[TXT]]
10                                            [[CSV]]
11                                        [[Details]]
12                                            [[TXT]]
13                                            [[CSV]]
14                                        [[Details]]
15                            [[Historical Archives]]
16                                            [[TXT]]
17                                            [[CSV]]
18                          

In [86]:
links[10]

<a href="ftp/F-F_Research_Data_Factors_daily_CSV.zip"><font color="#336699" face="Times New Roman, Times, serif" size="1">CSV</font></a>

In [108]:
def find_factors(x, frequency="daily", factor=3, format="csv"):
    try:
        x = x["href"]
    except:
        return False
    is_fff = False
    found_factor = False
    found_frequency = False
    found_format = False
    if frequency == "monthly":
        frequency = ""
    
    if factor == 3:
        factor = "data_factor"
    elif factor == 5:
        factor = "data_5_factor"

    x = x.lower()
    x = x.replace("2x3", "")
    if "research_data" in x:
        is_fff = True
    if frequency in x:
        found_frequency = True
    if factor in x:
        found_factor = True
    if format in x:
        found_format = True
    if is_fff and found_factor and found_frequency and found_format:
        return True
    else:
        return False

In [116]:
links[links.apply(lambda x: find_factors(x, frequency="weekly", factor=3, format="txt"))].iloc[0]["href"]

'ftp/F-F_Research_Data_Factors_weekly_TXT.zip'

In [117]:
def find_url(urls, frequency="daily", factor=3, format="csv"):
    filter_links = urls[urls.apply(lambda x: find_factors(x, frequency=frequency, factor=factor, format=format))]
    if len(filter_links) == 0:
        return None
    else:
        return filter_links.iloc[0]["href"]

In [119]:
find_url(links, frequency="weekly", factor=3, format="csv")

'ftp/F-F_Research_Data_Factors_weekly_CSV.zip'

In [120]:
url

'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html'

In [134]:
zip_url = url.split("data_library.html")[0] + find_url(links, frequency="weekly", factor=3, format="csv")
zip_url

'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_weekly_CSV.zip'

In [123]:
with open("fff3_weekly_raw.zip", "wb") as f:
    f.write(requests.get(zip_url).content)

In [132]:
import zipfile

with zipfile.ZipFile("fff3_weekly_raw.zip", "r") as zip_ref:
    zip_ref.extractall(".")
    print(zip_ref.namelist()[0])

F-F_Research_Data_Factors_weekly.csv


In [133]:
import os

In [135]:
r = requests.get(zip_url)

In [136]:
file_name = "fffff.csv"

In [137]:
with open(".fff.zip", "wb") as f:
        f.write(r.content)
        with zipfile.ZipFile(".fff.zip", "r") as zip_ref:
            zip_ref.extractall()
            old_name = zip_ref.namelist()[0]
            os.rename(old_name, file_name)
        os.remove(".fff.zip")

In [1]:
from pystock.FFF import FamaFrenchFactors

In [2]:
fff = FamaFrenchFactors()

In [5]:
fff.download(frequency="daily", factors=5, dir="fff")

File already exists. Use load() to load the file. Or set overwrite=True to overwrite the file.


In [12]:
df = fff.load(frequency="quarterly", factors=5, dir="fff", preprocess=True)

In [13]:
df

,Mkt-RF,SMB,HML,RMW,CMA,RF
1963-09-30,-0.0060,0.0021,0.0008,0.0024,0.0013,0.00014
1963-12-31,0.0056,0.0044,-0.0009,0.0017,-0.0022,0.00014
1964-03-31,-0.0019,0.0021,-0.0030,0.0001,-0.0002,0.00015
1964-06-30,0.0003,-0.0009,-0.0012,0.0030,-0.0034,0.00014
1964-09-30,-0.0003,0.0018,-0.0009,0.0008,0.0025,0.00013
...,...,...,...,...,...,...
2021-09-30,-0.0102,0.0008,-0.0078,-0.0123,-0.0036,0.00000
2021-12-31,-0.0031,0.0017,0.0053,0.0036,0.0041,0.00000
2022-03-31,-0.0155,0.0063,-0.0044,-0.0022,0.0040,0.00000
2022-06-30,-0.0096,0.0051,-0.0014,0.0089,-0.0016,0.00003


In [7]:
def convert_to_datetime(x):
    return datetime.datetime.strptime(x, "%Y%m%d")

In [11]:
df.index = df.index.astype(str)
df.index = pd.Series(df.index).apply(lambda x : pd.to_datetime(x, format="%Y%m%d"))
df

,Mkt-RF,SMB,HML,RF
1926-07-02,1.60,-0.62,-0.83,0.056
1926-07-10,0.36,-0.88,0.31,0.056
1926-07-17,1.01,0.59,-1.44,0.056
1926-07-24,-2.05,0.10,-0.18,0.056
1926-07-31,3.04,-1.82,-0.90,0.056
...,...,...,...,...
2022-10-28,3.97,1.31,0.70,0.058
2022-11-04,-3.56,-0.84,4.55,0.074
2022-11-11,5.85,0.15,-3.33,0.074
2022-11-18,-1.00,-1.42,0.76,0.074


In [14]:
pd.to_datetime(df.index)

DatetimeIndex(['1926-07-02', '1926-07-10', '1926-07-17', '1926-07-24',
               '1926-07-31', '1926-08-07', '1926-08-14', '1926-08-21',
               '1926-08-28', '1926-09-03',
               ...
               '2022-09-23', '2022-09-30', '2022-10-07', '2022-10-14',
               '2022-10-21', '2022-10-28', '2022-11-04', '2022-11-11',
               '2022-11-18', '2022-11-25'],
              dtype='datetime64[ns]', length=5030, freq=None)

In [16]:
import getFamaFrenchFactors as gff

In [17]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')

In [18]:
ff3_monthly

,date_ff_factors,Mkt-RF,SMB,HML,RF
0,1926-07-31,0.0296,-0.0256,-0.0243,0.0022
1,1926-08-31,0.0264,-0.0117,0.0382,0.0025
2,1926-09-30,0.0036,-0.0140,0.0013,0.0023
3,1926-10-31,-0.0324,-0.0009,0.0070,0.0032
4,1926-11-30,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...,...
1152,2022-07-31,0.0957,0.0281,-0.0410,0.0008
1153,2022-08-31,-0.0377,0.0139,0.0031,0.0019
1154,2022-09-30,-0.0935,-0.0082,0.0003,0.0019
1155,2022-10-31,0.0783,0.0010,0.0805,0.0023


In [19]:
fff.download(frequency="monthly", factors=3, format="csv", dir="fff")

Download complete. File saved as fff/fff_monthly_3_factors.csv


'fff/fff_monthly_3_factors.csv'

In [20]:
fff_m = fff.load(frequency="monthly", factors=3, format="csv", dir="fff")

In [21]:
fff_m

,Mkt-RF,SMB,HML,RF
192607,2.96,-2.56,-2.43,0.22
192608,2.64,-1.17,3.82,0.25
192609,0.36,-1.40,0.13,0.23
192610,-3.24,-0.09,0.70,0.32
192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2017,21.51,-4.97,-13.51,0.80
2018,-6.95,-3.21,-9.73,1.83
2019,28.28,-6.10,-10.34,2.15
2020,23.66,13.17,-46.57,0.45


In [33]:
fff_m[fff_m.index == '  1927']

Index(['  1927'], dtype='object')

In [28]:
fff_m.index

Index(['192607', '192608', '192609', '192610', '192611', '192612', '192701',
       '192702', '192703', '192704',
       ...
       '  2012', '  2013', '  2014', '  2015', '  2016', '  2017', '  2018',
       '  2019', '  2020', '  2021'],
      dtype='object', length=1253)

In [50]:
fff_daily = fff.load(frequency="daily", factors=3, format="csv", dir="fff")

In [51]:
fff_daily.index = pd.to_datetime(fff_daily.index)

In [56]:
fff_monthly = fff_daily.asfreq("M", method="ffill")

In [57]:
fff_monthly

,Mkt-RF,SMB,HML,RF
1926-07-31,0.46,-0.12,-0.17,0.009
1926-08-31,0.58,-0.52,-0.04,0.010
1926-09-30,0.30,-0.22,0.05,0.009
1926-10-31,-0.25,0.21,0.22,0.013
1926-11-30,0.26,0.02,-0.49,0.013
...,...,...,...,...
2022-07-31,1.44,-0.96,0.45,0.004
2022-08-31,-0.74,0.28,-0.44,0.008
2022-09-30,-1.42,0.50,0.27,0.009
2022-10-31,-0.67,0.39,0.75,0.011


In [61]:
pd.to_numeric(fff_monthly["Mkt-RF"])*0.01

1926-07-31    0.0046
1926-08-31    0.0058
1926-09-30    0.0030
1926-10-31   -0.0025
1926-11-30    0.0026
               ...  
2022-07-31    0.0144
2022-08-31   -0.0074
2022-09-30   -0.0142
2022-10-31   -0.0067
2022-11-30    0.0312
Freq: M, Name: Mkt-RF, Length: 1157, dtype: float64